In [ ]:
!pip install PyPDF2
!pip install fuzzywuzzy
!pip install transformers
!pip install textract
import json

# data manipulation
import pandas as pd
import string
# normalize nested JSON files
from pandas.io.json import json_normalize
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import PyPDF2, urllib.request , nltk , textract
from io import BytesIO
import json
#import weasyprint
import pandas as pd
from pandas.io.json import json_normalize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
from fuzzywuzzy import fuzz
import json
import logging
import multiprocessing as mp
import numpy as np
import pandas as pd
import os
import re
import regex
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 5.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 98.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.9/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install html2text
import urllib.request as urllib2
import html2text


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
json_file_path = "/content/drive/MyDrive/RAjobs/OSU-Out-with-Thumbnails.json"

with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

In [ ]:
len(data), data[3]

In [ ]:
links = [x['link'] for x in data]

In [ ]:
links = pd.DataFrame(links)

In [ ]:
links

In [ ]:
USDA = []

def RequestHTML(url):
  response = requests.get(url)
  return response.text

TEST = []
N = 1546
# usda_count = np.zeros([N,1])
usda_count = []
for i in range(N):
  USDA.append([])
  usda_count.append(0)
# for i in range(N):
  url_to_scrape = links.iloc[i,0]
  html_document = RequestHTML(url_to_scrape)
  soup = BeautifulSoup(html_document, "html.parser")
  content = []
  for link in soup.find_all('a', attrs={'href': re.compile("http")}):
    content.append(link.get('href', []))
    if ('usda' in link.get('href', [])) and (not 'extension' in link.get('href',[])):
        usda_count[i] += 1
        USDA[i].append(link.get('href', []))

  content = list(filter(None,content))
  TEST.append(content)



In [ ]:
len(content), len(USDA), len(TEST)

(37, 1546, 1546)

In [ ]:
links['usda_count'] = np.nan
links['usda_link'] = np.nan
#links['usda_count'].iloc[:30] = usda_count
links['usda_count'] = usda_count
links['usda_link'] = USDA

In [ ]:
links.to_excel('/content/drive/MyDrive/RAjobs/links.xlsx',index = False)

In [ ]:
#This one scrape all hyperlinks
link1 = []
for i in links:
    url = i

    response = requests.get(url)
    soup= BeautifulSoup(response.text, "html.parser")

    for link in soup.find_all(attrs={'href': re.compile("http")}):
        link1.append(link.get('href'))


In [ ]:
urls = links[0]
grab = requests.get(urls)
soup = BeautifulSoup(grab.text, 'html.parser')

# opening a file in write mode
f = open("test1.txt", "w")
# traverse paragraphs from soup
for link in soup.find_all("a"):
   data = link.get('href')
   f.write(data)
   f.write("\n")

f.close()



In [ ]:
# This one I exclude pngs
pdf = []
for i in links:
    url = i

    response = requests.get(url)
    soup= BeautifulSoup(response.text, "html.parser")
    j = 0



    for link in soup.find_all(attrs={'href': re.compile("http")}):
        content=[]
        content.append(link.get('href', []))
        for urls in content:
            if ('.pdf' in urls):
                j+=1
                pdf.append(link.get('href'))
            else:
                pass

        #link1.append(link.get('href'))


In [ ]:
pdf = pd.DataFrame(pdf)
pdf.to_excel('/content/drive/MyDrive/RAjobs/OSU_pdfs.xlsx',index = False)

In [ ]:

PDF = pdf.values.tolist()

In [ ]:
agpr = []
fmla = []
land = []
fnlo = []
fpex = []
fmpc = []
crop= []
grst = []
acrg = []
cropan = []
prog = []
hgpg = []
cofd = []
cost = []
mkpr = []
brls = []
catl = []
lstk = []
dary = []
ckeg = []
hony = []
s = 0
e = 0
f = 0
for urls in PDF:
  if ('agpr' in urls):
    s+=1
    agpr.append(link.get('href'))
  else:
    if ('fmla' in urls):
      e+=1
      fmla.append(link.get('href'))
    else:
      if ('land08' in urls) or ('AgriLandVa' in urls):
        f+=1
        land.append(link.get('href'))
      else:
        if ('fnlo' in urls) or ('FarmLandIn' in urls):
          fnlo.append(link.get('href'))
        else:
          if ('fpex' in urls) or ('FarmProdEx' in urls):
            fpex.append(link.get('href'))
          else:
            if ('fmpc' in urls) or ('FarmComp' in urls):
              fmpc.append(link.get('href'))
            else:
              if ('crop' in urls):
                crop.append(link.get('href'))
              else:
                if ('grst' in urls):
                  grst.append(link.get('href'))
                else:
                  if ('acrg' in urls) or ('Acre' in urls):
                    acrg.append(link.get('href'))
                  else:
                    if ('CropProdSu' in urls):
                      cropan.append(link.get('href'))
                    else:
                      if ('prog' in urls):
                        prog.append(link.get('href'))
                      else:
                        if ('hgpg' in urls):
                          hgpg.append(link.get('href'))
                        else:
                          if ('cofd' in urls):
                            cofd.append(link.get('href'))
                          else:
                            if ('cost' in urls):
                              cost.append(link.get('href'))
                            else:
                              if ('mkpr' in urls):
                                mkpr.append(link.get('href'))
                              else:
                                if ('brls' in urls):
                                  brls.append(link.get('href'))
                                else:
                                  if ('catl' in urls):
                                    catl.append(link.get('href'))
                                  else:
                                    if ('lstk' in urls):
                                      lstk.append(link.get('href'))
                                    else:
                                      if ('dary' in urls):
                                        dary.append(link.get('href'))
                                      else:
                                        if ('ckeg' in urls):
                                          ckeg.append(link.get('href'))
                                        else:
                                          if ('hony' in urls) or ('Hone' in urls):
                                            hony.append(link.get('href'))
                                          else:
                                            pass






In [ ]:
print(agpr)
print(fmla)
print(land)
print(fnlo)
print(fpex)
print(fmpc)
print(crop)
print(grst)
print(acrg)
print(cropan)
print(prog)
print(hgpg)
print(cofd)
print(cost)
print(mkpr)
print(brls)
print(catl)
print(lstk)
print(dary)
print(ckeg)
print(hony)

In [ ]:
pdf.to_excel('/content/drive/MyDrive/RAjobs/OKU_pdf.xlsx',index = False)

In [ ]:
# This one I exclude pngs
png = []
pdf = []
extension = []
video = []
for i in links:
  url = i
  try:
      response = requests.get(url)
      soup= BeautifulSoup(response.text, "html.parser")
      j = 0
      p = 0
      q = 0
      f = 0
      for link in soup.find_all(attrs={'href': re.compile("http")}):
          content=[]
          content.append(link.get('href', []))
          for urls in content:
              if ('.png' in urls):
                  j+=1
                  png.append(link.get('href'))
              else:
                  if ('.pdf' in urls):
                    p+=1
                    pdf.append(link.get('href'))
                  else:
                    if ('extension' in urls):
                      q+=1
                      extension.append(link.get('href'))
                    else:
                      if ("youtube" in urls):
                        f+=1
                        video.append(link.get('href'))
                      else:
                        pass

  except urllib.error.HTTPError:
      pass

        #link1.append(link.get('href'))

In [ ]:
print(len(png))
print(len(pdf))
print(len(extension))
print(len(video))

6195
801
19927
15


In [ ]:
len(png)

6195

In [ ]:
len(pdf)

801

In [ ]:
usda = []



for i in links:
    url = i

    response = requests.get(url)
    soup= BeautifulSoup(response.text, "html.parser")
    j = 0



    for link in soup.find_all(attrs={'href': re.compile("http")}):
        content=[]
        content.append(link.get('href', []))
        for urls in content:
            if ('usda' in urls):
              if ('extension' in urls):
                pass
              else:
                j+=1
                usda.append(link.get('href'))


In [ ]:
len(usda)

91

In [ ]:
cornell = []



for i in links:
    url = i

    response = requests.get(url)
    soup= BeautifulSoup(response.text, "html.parser")
    j = 0



    for link in soup.find_all(attrs={'href': re.compile("http")}):
        content=[]
        content.append(link.get('href', []))
        for urls in content:
            if ('cornell' in urls):
              j+=1
              cornell.append(link.get('href'))
            else:
              pass


In [ ]:
usda_link = pd.DataFrame(usda)
usda_link.to_csv('/content/drive/MyDrive/RAjobs/usda.csv')

In [ ]:
nass = []



for i in links:
    url = i

    response = requests.get(url)
    soup= BeautifulSoup(response.text, "html.parser")
    j = 0



    for link in soup.find_all(attrs={'href': re.compile("http")}):
        content=[]
        content.append(link.get('href', []))
        for urls in content:
            if ('usda' in urls):
                if ('nass' in urls):
                  j+=1
                  nass.append(link.get('href'))
            else:
                pass


In [ ]:
len(nass)

10

In [ ]:
report = []



for i in links:
    url = i

    response = requests.get(url)
    soup= BeautifulSoup(response.text, "html.parser")
    j = 0



    for link in soup.find_all(attrs={'href': re.compile("http")}):
        content=[]
        content.append(link.get('href', []))
        for urls in content:
            if ('report' in urls):

                j+=1
                report.append(link.get('href'))
            else:
                pass

In [ ]:
len(report)

136

In [ ]:
len(link1)

69845

In [ ]:

extension = []



for i in links:
    url = i

    response = requests.get(url)
    soup= BeautifulSoup(response.text, "html.parser")
    j = 0



    for link in soup.find_all(attrs={'href': re.compile("http")}):
        content=[]
        content.append(link.get('href', []))
        for urls in content:
            if ('extension' in urls):

                j+=1
                extension.append(link.get('href'))
            else:
                pass

In [ ]:

for i in link1:
    try:
      page = urllib2.urlopen(i)
    except urllib.error.URLError:
        continue
    except ConnectionError:
        continue
    html_content = page.read()
    file = open('file_text.txt', 'wb')
    file.write(html_content)
    file.close()